In [4]:
import os
import gc
import sys
import json
import gzip
import glob
import itertools
import numpy as np
import pandas as pd
import sklearn.manifold
import matplotlib as mpl
import matplotlib.pyplot as plt

from math import comb
from tqdm import tqdm
from copy import deepcopy
from argparse import Namespace
from collections import OrderedDict
from functools import partial
from scipy.stats import multinomial

from ticodm.utils import *
from ticodm.spatial_interaction_model import ProductionConstrained
from ticodm.contingency_table_mcmc import ContingencyTableMarkovChainMonteCarlo
from ticodm.contingency_table import instantiate_ct, ContingencyTable2D

mpl.rcParams['agg.path.chunksize'] = 10000

In [5]:
%matplotlib inline

# AUTO RELOAD EXTERNAL MODULES
%load_ext autoreload
%autoreload 2

This notebook outlines key functionality of the ```ContingencyTable``` and ```ContingencyTableMarkovChainMonteCarlo``` classes that are used for constructing and sampling discrete origin-destination matrices (ODMs), respectively.

# Create a 2D table
Choose a specific number of rows $I$ and number of columns $J$ to create a table.
Rows usually correspond to number of origin locations while columns map to destination locations.

In [6]:
# Choose 2D table size
I,J = 5,6
# Randomly create a 2D discrete origin-destination matrix
table = np.random.randint(1,50,size=(I*J)).reshape((I,J)).astype('int32')

Initialise a contingency table (another name for discrete origin - destination matrix). Two types of constraints are provided: axes and cells.  The specified below constraint implies that:
1. All rowsums (sums over axis 1) are fixed. This can be the origin demand for a location choice problem.
2. The values of cells (0,1) and (4,2) are fixed. These are specifc OD-pairs.

In [7]:
# Initialise contingency table (discrete origin - destination matrix)
ct = ContingencyTable2D(
    table=table,
    constraints={
        "axes":[[1]],
        "cells":[(0,1),(4,2)]
    }
)

The above initialisation is equivalent to fixed the grand total of the table as follows

In [8]:
# Initialise contingency table (discrete origin - destination matrix)
ct = ContingencyTable2D(
    table=table,
    constraints={
        "axes":[[1],[0,1]],
        "cells":[(0,1),(4,2)]
    }
)

Note that simply writing ```"axes":[1]``` is going to throw an error.

In [9]:
# Initialise contingency table (discrete origin - destination matrix)
ct = ContingencyTable2D(
    table=table,
    constraints={
        "axes":[1],
        "cells":[(0,1),(4,2)]
    }
)

TypeError: object of type 'int' has no len()

We have now initialised a 5x6 ODM.

In [10]:
# View object
ct

5x6 ContingencyTable2(Config)

In [11]:
# Print table
ct.table

array([[17,  1, 21, 21, 40, 39],
       [ 5, 32, 40, 12, 39,  4],
       [31, 40, 27, 22, 24, 28],
       [ 4, 44, 37, 31, 38, 41],
       [49, 34, 44, 45, 43, 28]], dtype=int32)

In [12]:
# Print table dimensions
ct.dims

array([5, 6], dtype=uint8)

In [13]:
# Print margins
ct.residual_margins
# ct.margins

{(0,): array([106, 151, 169, 131, 184, 140], dtype=int32),
 (1,): array([139, 132, 172, 195, 243], dtype=int32),
 (0, 1): array([881], dtype=int32)}

Note that fixing the rowsums implies also fixing the grand total of the table indexed by a sum over both 0 and 1 axes.

In [14]:
# Print constrained axes
ct.constraints['constrained_axes']

[(0, 1), (1,)]

In [15]:
# Print margins for constrained axes
for ax in ct.constraints['constrained_axes']:
    print(ax,":",ct.residual_margins[ax],ct.residual_margins[ax].sum())

(0, 1) : [881] 881
(1,) : [139 132 172 195 243] 881


In [16]:
# Print constrained cells
ct.constraints['cells']

[(0, 1), (4, 2)]

In [17]:
# Print values of constrained cells
for c in ct.constraints['cells']:
    print(c,":",ct.table[c])

(0, 1) : 1
(4, 2) : 44


# Deterministic optimisation algorithms

A simple way of generating a table satisfying all constraints is to use iterative residual filling (similar to iterative proportional fitting). This can be done as follows

In [86]:
# Initialise contingency table (discrete origin - destination matrix)
ct = ContingencyTable2D(
    table=table,
    constraints={
        "axes":[[1]],
        "cells":[(0,1),(4,2)]
    }
)

In [93]:
# Obtain solution using mininum residual filling solver
table_solution = ct.table_iterative_residual_filling_solution()

In [99]:
table_solution

array([[106,   1,  32,   0,   0,   0],
       [  0,  63,  57,  12,   0,   0],
       [  0,  45,  33,  31,  36,  27],
       [  0,  25,   3,  71,  77,  19],
       [  0,  17,  44,  17,  71,  94]], dtype=int32)

In [94]:
# Ensure that table satisfies constraints
ct.table_margins_admissible(table_solution) and ct.table_cells_admissible(table_solution)

True

In [128]:
# Obtain solution using uniform filling solver
table_solution = ct.table_iterative_uniform_residual_filling_solution()

In [129]:
table_solution

array([[106,   1,  32,   0,   0,   0],
       [  0,  63,  57,  12,   0,   0],
       [  0,  45,  33,  31,  36,  27],
       [  0,  25,   3,  71,  20,  76],
       [  0,  17,  44,  17, 128,  37]], dtype=int32)

In [130]:
# Ensure that table satisfies constraints
ct.table_margins_admissible(table_solution) and ct.table_cells_admissible(table_solution)

True

# Markov Chain Monte Carlo sampling of tables